# 05-descr-stats-risk-scores
This notebook analyeses the three risk scores in terms of predictive power and formal fairness.
Results are given in Appendix B.4, Table 4 of Zezulka and Genin (2024).

Requirements:
- numpy==1.26.3
- pandas==2.1.4
- holisticai[all] 

## Config

In [35]:
import os
import numpy as np
import pandas as pd

from holisticai.bias.metrics import classification_bias_metrics
from holisticai.efficacy.metrics import classification_efficacy_metrics

In [24]:
# change directory
wd_path = 'C:\\Users\\Zezulka\\Documents\\01_PhD\\030-Projects\\2023-01_ALMP_LTU\\'
os.chdir(wd_path)

file_path = 'data\\ARCHIVE\\1203_ALMP_effects_risk_fairFemale_sim.csv'
# file = 'data\\1203_ALM_simulation_results.csv'

# load simulation data
df = pd.read_csv(file)
df.head()

,Unnamed: 0,ID,age,canton_french,canton_german,canton_italian,canton_moth_tongue,city,city_big,city_medium,...,sd_policy_Austrian_risk_score_eo_lower_1,sd_policy_Austrian_risk_score_eo_lower_2,sd_policy_Austrian_risk_score_eo_lower_3,sd_policy_Austrian_risk_score_eo_lower_4,sd_policy_Austrian_risk_score_eo_lower_5,sd_policy_Austrian_risk_score_if_lower_1,sd_policy_Austrian_risk_score_if_lower_2,sd_policy_Austrian_risk_score_if_lower_3,sd_policy_Austrian_risk_score_if_lower_4,sd_policy_Austrian_risk_score_if_lower_5
0,1,29410,49,0,1,0,0,3,1,0,...,0.000000,0.025522,0.025522,0.027284,0.027284,0.082857,0.099913,0.056883,0.048936,0.111881
1,2,2306,47,0,1,0,0,1,0,0,...,0.129951,0.112460,0.099804,0.115060,0.111635,0.106649,0.107784,0.104306,0.078342,0.111952
2,3,48710,42,0,1,0,0,1,0,0,...,0.089981,0.118963,0.094978,0.074377,0.090354,0.090889,0.095031,0.119094,0.108857,0.108307
3,4,9665,50,0,1,0,0,1,0,0,...,0.000000,0.034346,0.035054,0.028043,0.034346,0.093497,0.083167,0.060916,0.096911,0.042485
4,5,65086,55,0,1,0,0,1,0,0,...,0.000000,0.000000,0.005230,0.005230,0.004892,0.104249,0.075866,0.100223,0.087461,0.065925


In [25]:
# (non-) sensitive groups 
female = np.squeeze((df['female'] == 1).values)
male = np.squeeze((df['female'] == 0).values)

non_citizen = np.squeeze((df['swiss'] == 0).values)
citizen = np.squeeze((df['swiss'] == 1).values)

# Decision threshold for binary predictions
t = 0.5

# set ground truth variable
y_true = df['y_exit12']


## Predictive performance
Risk scores are binarized at decision threshold $t=0.5$.

In [26]:
# analysis
df_classification = classification_efficacy_metrics(y_true, (df['risk_score_log'] >= t).astype(int))
df_classification_sp = classification_efficacy_metrics(y_true, (df['risk_score_sp'] >= t).astype(int))
df_classification_eo = classification_efficacy_metrics(y_true, (df['risk_score_eo'] >= t).astype(int))

# prepare results
classification = pd.concat([df_classification, df_classification_sp, df_classification_eo], axis=1).iloc[:, [0,2,4,1]]
classification.columns = ['Baseline','SP','EO','Reference']

In [27]:
classification

,Baseline,SP,EO,Reference
Metric,,,,
Accuracy,0.643959,0.643959,0.644519,1
Balanced Accuracy,0.605870,0.608890,0.609357,1
Precision,0.612296,0.605352,0.606604,1
Recall,0.383559,0.404204,0.404129,1
F1-Score,0.471658,0.484739,0.485086,1


## Fairness analysis
Risk scores are binarized at decision threshold $t=0.5$.

In [28]:
# fairness analysis: group_a = female

fair_baseline = classification_bias_metrics(female, male, (df['risk_score_log'] >= t).astype(int), y_true)
fair_sp = classification_bias_metrics(female, male, (df['risk_score_sp'] >= t).astype(int), y_true)
fair_eo = classification_bias_metrics(female, male, (df['risk_score_eo'] >= t).astype(int), y_true)

# prepare results
fairness = pd.concat([fair_baseline, fair_sp, fair_eo], axis=1).iloc[:, [0,2,4,1]]
fairness.columns = ['Baseline','SP','EO','Reference']

In [29]:
fairness

,Baseline,SP,EO,Reference
Metric,,,,
Statistical Parity,0.116257,0.040623,0.019089,0
Disparate Impact,1.556759,1.156889,1.071307,1
Four Fifths Rule,0.642360,0.864387,0.933439,1
Cohen D,0.267515,0.090901,0.042711,0
2SD Rule,23.581403,8.074918,3.797135,0
Equality of Opportunity Difference,0.172736,0.069538,0.043985,0
False Positive Rate Difference,0.061541,0.005179,-0.013575,0
Average Odds Difference,0.117138,0.037359,0.015205,0
Accuracy Difference,0.019174,0.010068,0.010213,0


### Statistical Parity (Independence)

In [30]:
# baseline (logistic ridge regression)
p_0 = (df['risk_score_log'] >= t).astype(int)[female==0].mean()
p_1 = (df['risk_score_log'] >= t).astype(int)[female==1].mean()

print('Log:')
print('P(y_hat=1|A=1) - P(y_hat=1|A=0) = ' + str((p_1 - p_0)))
# print('P(y_hat=0|A=1) - P(y_hat=0|A=0) = ' + str(np.abs((1-p_1) - (1-p_0))))

# Statistical parity constraint
p_0 = (df['risk_score_sp'] >= t).astype(int)[female==0].mean()
p_1 = (df['risk_score_sp'] >= t).astype(int)[female==1].mean()

print('SP:')
print('P(y_hat=1|A=1) - P(y_hat=1|A=0) = ' + str(np.abs(p_1 - p_0)))

# Equalized odds constraint
p_0 = (df['risk_score_eo'] >= t).astype(int)[female==0].mean()
p_1 = (df['risk_score_eo'] >= t).astype(int)[female==1].mean()

print('EO:')
print('P(y_hat=1|A=1) - P(y_hat=1|A=0) = ' + str(np.abs(p_1 - p_0)))

Log:
P(y_hat=1|A=1) - P(y_hat=1|A=0) = 0.11625651992007055
SP:
P(y_hat=1|A=1) - P(y_hat=1|A=0) = 0.040622915626858935
EO:
P(y_hat=1|A=1) - P(y_hat=1|A=0) = 0.01908916255018739


### Separation Test
(1) Equal Opportunity

(2) False Positive Parity

In [31]:
# baseline (logistic ridge regression)
p_1_tpr = (df['risk_score_log'] >= t).astype(int)[(df['y_exit12'] == 1) & (df['female'] == 1)].mean()
p_0_tpr = (df['risk_score_log'] >= t).astype(int)[(df['y_exit12'] == 1) & (df['female'] == 0)].mean()

p_1_fpr = (df['risk_score_log'] >= t).astype(int)[(df['y_exit12'] == 0) & (df['female'] == 1)].mean()
p_0_fpr = (df['risk_score_log'] >= t).astype(int)[(df['y_exit12'] == 0) & (df['female'] == 0)].mean()

print('Log:')
print('P(y_hat=1|A=1,Y=1) - P(y_hat=1|A=0,Y=1) = ' + str(p_1_tpr - p_0_tpr))
print('P(y_hat=1|A=1,Y=0) - P(y_hat=1|A=0,Y=0) = ' + str(p_1_fpr - p_0_fpr))

# Statistical parity constraint
p_1_tpr = (df['risk_score_sp'] >= t).astype(int)[(df['y_exit12'] == 1) & (df['female'] == 1)].mean()
p_0_tpr = (df['risk_score_sp'] >= t).astype(int)[(df['y_exit12'] == 1) & (df['female'] == 0)].mean()

p_1_fpr = (df['risk_score_sp'] >= t).astype(int)[(df['y_exit12'] == 0) & (df['female'] == 1)].mean()
p_0_fpr = (df['risk_score_sp'] >= t).astype(int)[(df['y_exit12'] == 0) & (df['female'] == 0)].mean()

print('SP:')
print('P(y_hat=1|A=1,Y=1) - P(y_hat=1|A=0,Y=1) = ' + str(p_1_tpr - p_0_tpr))
print('P(y_hat=1|A=1,Y=0) - P(y_hat=1|A=0,Y=0) = ' + str(p_1_fpr - p_0_fpr))

# Equalized odds constraint
p_1_tpr = (df['risk_score_eo'] >= t).astype(int)[(df['y_exit12'] == 1) & (df['female'] == 1)].mean()
p_0_tpr = (df['risk_score_eo'] >= t).astype(int)[(df['y_exit12'] == 1) & (df['female'] == 0)].mean()

p_1_fpr = (df['risk_score_eo'] >= t).astype(int)[(df['y_exit12'] == 0) & (df['female'] == 1)].mean()
p_0_fpr = (df['risk_score_eo'] >= t).astype(int)[(df['y_exit12'] == 0) & (df['female'] == 0)].mean()

print('EO:')
print('P(y_hat=1|A=1,Y=1) - P(y_hat=1|A=0,Y=1) = ' + str(p_1_tpr - p_0_tpr))
print('P(y_hat=1|A=1,Y=0) - P(y_hat=1|A=0,Y=0) = ' + str(p_1_fpr - p_0_fpr))

Log:
P(y_hat=1|A=1,Y=1) - P(y_hat=1|A=0,Y=1) = 0.17273590437302494
P(y_hat=1|A=1,Y=0) - P(y_hat=1|A=0,Y=0) = 0.06154068490942541
SP:
P(y_hat=1|A=1,Y=1) - P(y_hat=1|A=0,Y=1) = 0.06953847406477615
P(y_hat=1|A=1,Y=0) - P(y_hat=1|A=0,Y=0) = 0.0051793426603745085
EO:
P(y_hat=1|A=1,Y=1) - P(y_hat=1|A=0,Y=1) = 0.04398486031491289
P(y_hat=1|A=1,Y=0) - P(y_hat=1|A=0,Y=0) = -0.013575486223437655


### Sufficiency Test
(1) Negative Predictive Parity 

(2) Positive Predictive Parity

In [32]:
# baseline (logistic ridge regression)
p_a1y0 = 1 - df['y_exit12'][(df['female'] == 1) & ((df['risk_score_log'] >= t).astype(int) == 0)].mean()
p_a0y0 = 1 - df['y_exit12'][(df['female'] == 0) & ((df['risk_score_log'] >= t).astype(int) == 0)].mean()

p_a1y1 = df['y_exit12'][(df['female'] == 1) & ((df['risk_score_log'] >= t).astype(int) == 1)].mean()
p_a0y1 = df['y_exit12'][(df['female'] == 0) & ((df['risk_score_log'] >= t).astype(int) == 1)].mean()

print('Log:')
print('P(y=1|A=1,y_hat=0) - P(y=1|A=0,y_hat=0) = ' + str(np.abs(p_a1y0 - p_a0y0)))
print('P(y=1|A=1,y_hat=1) - P(y=1|A=0,y_hat=1) = ' + str(np.abs(p_a1y1 - p_a0y1)))

# Statistical parity constraint
p_a1y0 = 1 - df['y_exit12'][(df['female'] == 1) & ((df['risk_score_sp'] >= t).astype(int) == 0)].mean()
p_a0y0 = 1 - df['y_exit12'][(df['female'] == 0) & ((df['risk_score_sp'] >= t).astype(int) == 0)].mean()

p_a1y1 = df['y_exit12'][(df['female'] == 1) & ((df['risk_score_sp'] >= t).astype(int) == 1)].mean()
p_a0y1 = df['y_exit12'][(df['female'] == 0) & ((df['risk_score_sp'] >= t).astype(int) == 1)].mean()

print('SP:')
print('P(y=1|A=1,y_hat=0) - P(y=1|A=0,y_hat=0) = ' + str(np.abs(p_a1y0 - p_a0y0)))
print('P(y=1|A=1,y_hat=1) - P(y=1|A=0,y_hat=1) = ' + str(np.abs(p_a1y1 - p_a0y1)))

# Equalized odds constraint
p_a1y0 = 1 - df['y_exit12'][(df['female'] == 1) & ((df['risk_score_eo'] >= t).astype(int) == 0)].mean()
p_a0y0 = 1 - df['y_exit12'][(df['female'] == 0) & ((df['risk_score_eo'] >= t).astype(int) == 0)].mean()

p_a1y1 = df['y_exit12'][(df['female'] == 1) & ((df['risk_score_eo'] >= t).astype(int) == 1)].mean()
p_a0y1 = df['y_exit12'][(df['female'] == 0) & ((df['risk_score_eo'] >= t).astype(int) == 1)].mean()

print('EO:')
print('P(y=1|A=1,y_hat=0) - P(y=1|A=0,y_hat=0) = ' + str(np.abs(p_a1y0 - p_a0y0)))
print('P(y=1|A=1,y_hat=1) - P(y=1|A=0,y_hat=1) = ' + str(np.abs(p_a1y1 - p_a0y1)))

Log:
P(y=1|A=1,y_hat=0) - P(y=1|A=0,y_hat=0) = 0.01115312187783113
P(y=1|A=1,y_hat=1) - P(y=1|A=0,y_hat=1) = 0.061569190176363864
SP:
P(y=1|A=1,y_hat=0) - P(y=1|A=0,y_hat=0) = 0.011270600947347553
P(y=1|A=1,y_hat=1) - P(y=1|A=0,y_hat=1) = 0.07249432993750904
EO:
P(y=1|A=1,y_hat=0) - P(y=1|A=0,y_hat=0) = 0.015842253303519738
P(y=1|A=1,y_hat=1) - P(y=1|A=0,y_hat=1) = 0.0814027177143326
